# AdventOfCode 2017

#### Arnaud Delaunay

## Utils

In [1]:
import sys
sys.setrecursionlimit(100000)
import numpy as np

def get_inp(day, mode='np'):
    if mode=='np':
        try:
            return np.loadtxt('inputs/day%s' % day)
        except:
            return open('inputs/day%s' % day).read()[:-1]
    return open('inputs/day%s' % day).read()[:-1]
        

outputs = {}

def reload_answers(day):
    outputs[day]['answers'] = outputs[day]['func'](day)
    return outputs[day]['answers']

def add_day_func_to_dict(day_func, day, mode='np', callback=None):
    inp = get_inp(day, mode=mode)
    if callback:
        inp = callback(inp)
    f = lambda day : day_func(inp)
    outputs[day] = {
        'func' : f,
        'answers' : f(day)
    }
    return outputs[day]['answers']

# Day 1

In [62]:
def get_output(inp, i, add):
    return int(inp[i]) if inp[i]==inp[np.mod(i+add,len(inp))] else 0

def day_1(inp):
    S1 = 0
    S2 = 0
    for i in range(len(inp)):
        S1+=get_output(inp,i,1)
        S2+=get_output(inp,i,len(inp)/2)
    print S1,S2

In [63]:
add_day_func_to_dict(day_1, '1', mode=0)

1119 1420


# Day 2

In [64]:
def day_2_1(inp):
    return int(np.array(map(lambda x : x.max() - x.min(), inp)).sum())

def day_2_2(inp):
    def get_div(row):
        for i, e in enumerate(row):
            for e2 in np.hstack((row[:i],row[i+1:])):
                if e%e2==0:
                    return e/e2
    return int(np.array(map(get_div, inp)).sum())

day_2 = lambda inp : (day_2_1(inp),day_2_2(inp))

In [65]:
add_day_func_to_dict(day_2, '2', mode='np')

(45351, 275)

# Day 3

In [74]:
#DAY 3.1
def get_path_to_1(inp):
    k = int(np.sqrt(inp))
    K = k/2 - (1-k%2)
    I = k%2
    p = inp-(k*k)
    return abs(K-min(p-1, k))+abs(K+I-max(p-k-1,0))

# %time get_path_to_1(inp)

#DAY 3.2

#USELESS FINALLY
def get_coordinates(inp):
    if inp == 1:
        return 0,0
    k = int(np.sqrt(inp))
    K = k/2
    p = inp-(k*k)
    add = 1 if p==0 else 0 # if it's a square
    I = (-1)**(k%2+1)*(K-min(p-1,k)-add)
    J = (-1)**(k%2+1)*(K+k%2-max(p-k-1,0)-add)
    return I, J

def get_idx(I,J):
    if (I,J) == (0,0):
        return 1
    sign = np.sign(J-I) if J!=I else -1 #we want sign(0)=-1
    K = max(sign*J, -sign*I)-1
    k = K*2+1-min(0,sign)
    N = K +k**2+1
    N += -sign*I - 1*(min(sign, 0)) if -sign*I<=sign*J else -sign*J + k + 1
    return int(N)

M = {}
def compute_sum(I,J):
    if I==0 and J==0:
        return 1
    if '%d,%d' % (I,J) in M:
        return M['%d,%d' % (I,J)]
    this_idx = get_idx(I,J)
    S = 0
    for i in range(-1,2):
        for j in range(-1,2):
            if get_idx(I+i,J+j)<this_idx:
                if '%d,%d' % (I+i,J+j) not in M:
                    M['%d,%d' % (I+i,J+j)] = compute_sum(I+i,J+j)
                S+=M['%d,%d' % (I+i,J+j)]
    return S

def find_smallest_larger_than(X):
    k = 0
    while(True):
        if compute_sum(k,k)>X:
            sign=1
            if compute_sum(-k,-k)<X:
                sign = -1 
            value = compute_sum(sign*(k-1),sign*k)
            s = 2
            while X>value:
                value = compute_sum(sign*(k-min(s,2*k)),sign*(k-max(s-2*k,0)))
                s+=1
            return value
        k += 1
        
# %time find_smallest_larger_than(10211726171726281982716820)

day_3  = lambda inp : (get_path_to_1(inp), find_smallest_larger_than(inp))
add_day_func_to_dict(day_3, '3', mode='np')

(371.0, 369601)

# DAY 4

In [81]:
def check_passphrase(paph):
    L = paph.split(' ')
    return len(L)==len(set(L))
def day_4_1(inp):
    return sum(map(check_passphrase,inp))

#inp = get_inp('4').split('\n')[:-1]
#day_4_1(inp)

def doublecheck_passphrase(paph):
    L = map(lambda x : set(x), paph.split(' '))
    unique = reduce(lambda l, x: l.append(x) or l if x not in l else l, L, [])
    return len(unique)==len(L)

def day_4_2(inp):
    return sum(map(doublecheck_passphrase,inp))

#inp = get_inp('4').split('\n')[:-1]
#day_4_2(inp)

day_4 = lambda inp : (day_4_1(inp), day_4_2(inp))

In [82]:
add_day_func_to_dict(day_4, '4', mode=0, callback=lambda x : x.split('\n'))

(386, 208)

# DAY 5

In [98]:
def get_number_of_steps(this_inp, change, change2_if_condition=1, condition=3):
    inp = this_inp.copy()
    cursor = 0
    L = len(inp)
    step = 0
    while True:
        prev = cursor
        cursor += inp[cursor]
        inp[prev] += change2_if_condition if inp[prev]>=condition else change
        step +=1
        if cursor >= L or cursor <0:
            break
    return step

day_5 = lambda inp : (get_number_of_steps(inp, 1), get_number_of_steps(inp, 1, -1, 3))

In [99]:
add_day_func_to_dict(day_5, '5')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(396086, 28675390)

# DAY 6

In [19]:
import multiprocessing
from functools import partial

def day_6(inp):
    L=len(inp)
    states =[] 

    def choose_bank(inp):
        return inp.argmax()

    def get_spread(idxn_b, iv):
        i, v = iv
        idx, n_b = idxn_b
        # print i, v, idx, n_b
        if i == idx:
            v = 0
        S=v+n_b/L
        if np.mod((i-idx)-1, L)<(n_b%L):
            S+=1
        return S

    def spread_blocks(idx, inp):
        n_b = inp[idx] 
        #pool = multiprocessing.Pool()
        #inp = pool.map(partial(get_spread, (idx,n_b)), enumerate(inp))
        inp = map(partial(get_spread, (idx,n_b)), enumerate(inp))
        return np.array(inp) 

    step = 0
    while True:
        idx = choose_bank(inp) 
        inp = spread_blocks(idx, inp)
        hashinp = hash(inp.tostring())
        step += 1
        if hashinp in states:
            break
        states.append(hashinp)

    return (step, len(states)-states.index(hashinp))

In [20]:
add_day_func_to_dict(day_6, '6', mode='np', callback=lambda x : x.astype(int))

(12841, 8038)

# DAY 7

In [14]:
def get_root(inp):
    inp = map(lambda x : x.split(' '), inp)
    leafs = dict([(l[0], {'w' : int(l[1].replace('(','').replace(')',''))}) for l in inp if '->' not in l])
    roots = dict([
        (l[0], {
                'w' : int(l[1].replace('(','').replace(')','')),
                'sons' : map(lambda x : x.replace(',',''), l[l.index('->')+1:])
               }) for l in inp if '->' in l])
    leafs_of_roots = [leaf for word in roots for leaf in roots[word]['sons']]
    for root in roots:
        if root not in leafs_of_roots:
            return root, roots, leafs

answer_7_1 = lambda inp : get_root(inp)[0]

def get_weights_of_sons(word, leafs, roots):
    if word in leafs:
        return leafs[word]['w']
    else:
        return np.sum([get_weights_of_sons(w, leafs, roots) for w in roots[word]['sons']]) + roots[word]['w']

def get_unbalanced_son(word, leafs, roots):
    if word in leafs:
        return None, None, None
    L = [get_weights_of_sons(w, leafs, roots) for w in roots[word]['sons']]
    value = len(set(L))==1
    if value:
        return None, None, None
    else:
        for i, l in enumerate(L):
            if L.count(l)==1:
                return roots[word]['sons'][i], l, L[np.mod((i+1),len(L))]

def answer_7_2(inp):
    word, roots, leafs = get_root(inp)
    should_be = 0
    v = 0
    while word is not None:
        prev_word, diff = (word, v-should_be)
        word, should_be, v = get_unbalanced_son(word, leafs, roots)
    #print "Bad value is %d, should be %d (weight of word %s)" % (roots[prev_word]['w'], roots[prev_word]['w'] + diff, prev_word)
    return roots[prev_word]['w'] + diff

day_7 = lambda inp : (answer_7_1(inp), answer_7_2(inp))

In [15]:
add_day_func_to_dict(day_7, '7', mode=0, callback=lambda x : x.split('\n'))

Bad value is 428, should be 420 (weight of word drjmjug)


('eugwuhl', 420)

# DAY 8

In [21]:
import operator
import numpy as np

def day_8(inp):
    ACTIONS = {'inc' : 1, 'dec' : -1}
    CONDITIONS = {"==" : lambda x,y : x==y,
                  "!=" : lambda x,y : x!=y,
                  "<=" : lambda x,y : x<=y,
                  ">=" : lambda x,y : x>=y,
                  ">" : lambda x,y : x>y,
                  "<" : lambda x,y : x<y
                 }
    variables = {'max_value' : -np.inf}
    
    def code_parser(line):
        """
        <inp_variable> <action> <var_value> if <cond_variable> <condition> <cond_value>
        """
        inp_variable, action, var_value, _, cond_variable, condition, cond_value = line.split()
        variables[inp_variable] = 0 if inp_variable not in variables else variables[inp_variable]
        variables[cond_variable] = 0 if cond_variable not in variables else variables[cond_variable]
        if CONDITIONS[condition](variables[cond_variable], int(cond_value)):
            variables[inp_variable] = variables[inp_variable] + ACTIONS[action]*int(var_value)
            variables['max_value'] = max(variables['max_value'], variables[inp_variable])

    map(code_parser, inp)
    max_value = variables['max_value']
    del variables['max_value']
    return(max(variables.itervalues()), max_value)

In [22]:
add_day_func_to_dict(day_8, '8', mode=0, callback=lambda x : x.split('\n'))

(4567, 5636)

# DAY 9

In [26]:
import re
def remove_char(inp):
    return re.sub('!.{1}', '', inp)

def remove_garbage(inp):
    return re.sub('<.*?>', '', inp)

def count_in_garbage(inp):
    myre = re.compile(r'<.*?>')
    return np.sum(len(x) for x in map(lambda x : x[1:-1], myre.findall(inp)))

def get_subgroups(inp):
    return inp[1:-1].split(',')

def parenthetic_contents(string):
    stack = []
    for i, c in enumerate(string):
        if c == '{':
            stack.append(i)
        elif c == '}' and stack:
            start = stack.pop()
            yield (len(stack), string[start + 1: i])
            
day_9_1 = lambda inp : np.sum([depth+1 for depth, val in parenthetic_contents(remove_garbage(remove_char(inp)))])
day_9_2 = lambda inp : count_in_garbage(remove_char(inp))

day_9 = lambda inp : (day_9_1(inp), day_9_2(inp))

In [27]:
add_day_func_to_dict(day_9, '9', mode=0)

(14204, 6622)

# DAY 10

In [35]:
# DAY 10.1
def modify_list(L, current_position, length, skip_size):
    N = (current_position+length)
    number_of_first_elements = max(0,N-len(L))
    sub_list = L[current_position:N] + L[:number_of_first_elements]
    sub_list.reverse()
    L[current_position:N] = sub_list[:(length-number_of_first_elements)]
    L[:number_of_first_elements] = sub_list[length-number_of_first_elements:]
    return L, np.mod((current_position+length+skip_size),len(L)), skip_size+1

def day_10_1(inp):
    L = range(256)
    current_position = 0
    skip_size = 0
    for length in inp:
        L, current_position, skip_size = modify_list(L, current_position, length, skip_size)
    return L[0]*L[1]

In [38]:
inp = get_inp('10')[:-1]

def to_ascii(string):
    return [ord(c) for c in string]

def get_lengths(string):
    return to_ascii(string)+[17, 31, 73, 47, 23]

def get_sparse_hash(string):
    L = range(256)
    current_position = 0
    skip_size = 0
    lengths = get_lengths(string)
    for length in lengths*64:
        L, current_position, skip_size = modify_list(L, current_position, length, skip_size)
    return L

def perform_xor(list_of_nums):
    S = 0
    for x in list_of_nums:
        S ^= x
    return S

def to_dense(sparse):
    return ''.join(
        map(
            lambda x : hex(x)[2:] if len(hex(x)[2:])==2 else '0%s' % hex(x)[2:],
            map(
                lambda sixteens : perform_xor(sixteens),
                [sparse[16*i:16*(i+1)] for i in range(16)]
            )
        ))

def day_10_2(inp):
    inp = ','.join(map(str,inp))
    return to_dense(get_sparse_hash(inp))

day_10 = lambda inp : (day_10_1(inp), day_10_2(inp))

In [39]:
add_day_func_to_dict(day_10, '10', mode=0, callback=lambda x : map(int, x.split(',')))

(1935, 'dc7e7dee710d4c7201ce42713e6b8359')

# DAY 11

In [40]:
def day_11(inp):
    moves = {
        's' : (2,0),
        'n' : (-2,0),
        'se' : (1, 1),
        'ne' : (-1,1),
        'sw' : (1,-1),
        'nw' : (-1,-1)
    }

    def add(pos, move):
        return (pos[0]+move[0], pos[1]+ move[1])

    def diff(pos, start):
        return (pos[0]-start[0], pos[1]-start[1])

    def compute_steps(pos):
        if abs(pos[1])>abs(pos[0]):
            s = abs(pos[1])
        else:
            s = abs(pos[1])+(abs(pos[0])-abs(pos[1]))/2
        return s

    pos = (0,0)
    steps = []
    poss = []
    for move in inp:
        pos = add(pos, moves[move])
        poss.append(pos)
        steps.append(compute_steps(pos))
    return (steps[-1], max(steps))

In [41]:
add_day_func_to_dict(day_11, '11', mode=0, callback=lambda x : x.split(','))

(664, 1447)

# DAY 12

In [42]:
def parser(line, neighbors):
    idx, neighs = line.split(' <-> ')
    idx = int(idx)
    neighs = map(int, neighs.split(','))
    if idx not in neighbors:
        neighbors[idx] = set()
    map(lambda n : neighbors[idx].add(n), neighs)
    return neighs, neighbors

def get_all_neighs_of_idx(inp, idx, neighbors):
    all_neighs = set([idx])
    direct_neighs, neighbors = parser(inp[idx], neighbors)
    map(lambda n : all_neighs.add(n), direct_neighs)
    for new_id in direct_neighs:
        if new_id not in neighbors:
            this_neighs, neighbors = get_all_neighs_of_idx(inp, new_id, neighbors)
            map(lambda n : all_neighs.add(n), this_neighs)
    return all_neighs, neighbors

def get_all_groups(inp):
    neighbors = {}
    groups = []
    ids_in_groups = set()
    for idx in range(len(inp)):
        if idx not in ids_in_groups:
            new_group, neighbors = get_all_neighs_of_idx(inp, idx, neighbors)
            ids_in_groups = ids_in_groups.union(new_group)
            groups.append(new_group)
    return groups

""" TESTS
test = ["0 <-> 2", "1 <-> 1","2 <-> 0, 3, 4","3 <-> 2, 4","4 <-> 2, 3, 6","5 <-> 6","6 <-> 4, 5"]group_test, _ = get_all_neighs_of_idx(test, 0, {})
groups = get_all_groups(test)
groups
"""

def day_12(inp):
    group_real, _ = get_all_neighs_of_idx(inp, 0, {})
    groups = get_all_groups(inp)
    return (len(group_real), len(groups))

In [43]:
add_day_func_to_dict(day_12, '12', mode=0, callback=lambda x : x.split('\n'))

(130, 189)

# Day 13

In [103]:
def get_pos_of_scanner(rang, t):
    rest = np.mod(t,2*rang-2)
    return rest if rest<rang else np.mod(-rest+rang-1,rang-1)
    
def get_layers(inp):
    return map(lambda l : (int(l.split(': ')[0]), int(l.split(': ')[1])), inp)
    
def is_caught_on_layer(layer, delay=0):
    return (True, layer[0]*layer[1]) if get_pos_of_scanner(layer[1], layer[0]+delay)==0 else (False, 0)

def day_13_1(inp, delay=0):
    layers = get_layers(inp)
    return np.sum(map(lambda layer : is_caught_on_layer(layer, delay=delay)[1], layers))

def day_13_2(inp):
    layers = get_layers(inp)
    delay = 1
    while True:
        caught = False
        for layer in layers:
            caught = is_caught_on_layer(layer, delay=delay)[0]
            if caught:
                break
        if not caught:
            return delay
        delay+=1

day_13 = lambda inp : (day_10_1(inp), day_10_2(inp))

In [105]:
add_day_func_to_dict(day_13, '13', mode=0, callback=lambda x : x.split('\n')[:-1])

(1900, 3966414)